# Dynamic Programming (動的計画法)

競プロにて頻繁に登場するアルゴリズム  
簡単な問題を通じて実装をしていく

Reference
- 競技プログラミングの鉄則
    - 書籍。図が多くて分かりやすい。

動的計画法は一言で表すと、より小さい問題の結果を利用して問題を解く方法などとよく言われる。  
が、そういわれても(自分は)ピンとこないので実際に具体例を見る方が多分手っ取り早い。

## 例題1 動的計画法の基本
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_p

---
以下解法とか、考え方とか

- dpの作り方  
dp[i]を部屋1から部屋iまでの最短時間とする  
    - 今回のような簡単な例では自然に思いつきやすいが、ここのdpをどう作るかが難しく色々なパターンがある  
- dp[i]の計算の仕方  
dp[1]は移動する必要がないから0(部屋は1-index)  
dp[2]は1から$A_2$時間かけて移動するしかないのでdp[2]=$A_2$
dp[3]は部屋1から$A_2 + A_3$かけて移動するか、$B_3$かけて移動するかの2通り  
もっと言うと1個前の部屋からの移動時間dp[2] + $A_3$か2個前部屋からの移動時間dp[1] + $B_3$と表せる  
一般化すると部屋iは1個前の部屋からdp[i-1] + $A_i$かけて移動するか、dp[i-2] + $B_i$かけて移動するかの2通りとなる  
    -  dpは最短時間を計算してあるはずなので考えるのはこの2通りだけで、それ以前は考えなくて大丈夫  
従って
$$
dp[1] = 0  \\
dp[2] = A_2 \\
dp[i] = \min (dp[i-1] + A_i, dp[i-2] + B_i) \\
$$
と表せ、これを手前から(0からN方向へ)計算していけば計算量$O(N)$で求まる


In [1]:
# 回答例

# 入力
N = 5
a = [2,4,1,3]
b = [5,3,7]

# 回答
dp = [] # numpy使ってdp = np.zeros(shape=(N,))とかしてdp[i] = の形にしても良い。そっちの方が分かりやすい
# 以下0-index (分かりやすくしたいなら1-indexになるよう番兵とか入れて調整して)
dp.append(0) # dp[1] = 0
dp.append(a[0]) # dp[2] = A_2
for i in range(2,N):
    dp.append(min(dp[i-1]+a[i-1],dp[i-2]+b[i-2]))

print(dp[-1])

8


In [2]:
# 添え字がバチクソ分かりにくいからnumpyの配列を使うバージョンも
# ついでに問題に合わせて1-indexにしちゃう
import numpy as np
dp = np.zeros(shape=(N+1,),dtype=np.int64)
a = [0,0] + a
b = [0,0,0] + b

dp[0] = 1e9
dp[1] = 0
dp[2] = a[2]
for i in range(3,N+1):
    dp[i] = min(dp[i-1]+a[i],dp[i-2]+b[i])
print(dp[-1])


8


## 例題2 動的計画法の復元
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_q

---
先の問題では手前から順にdp[i]を更新していった  
動的計画法の経路の復元では逆にゴール側からdp[i]を計算するときにどっちのルートからやってきたかを辿ることで復元を行う  
具体的には  
dp[N]はd[N-1] + $A_N$ or d[N-2] + $B_N$を計算することで合う方がdp[N]の1つ手前の状態であることが分かる  
d[N-2] + $B_N$ = dp[N]なら1つ前はN-2なので今度はdp[N-2]はdp[N-2-1] + $A_{N-2}$ or dp[N-2-2] + $B_{N-2}$を計算することで
どっちのルートからやってきたか分かる  
これを続けてスタートまで求めることで経路の復元ができる

In [3]:
# 回答例
# 入力
N = 5
a = [2,4,1,3]
b = [5,3,7]

import numpy as np
# 動的計画法の計算
dp = np.zeros(shape=(N+1,),dtype=np.int64)
a = [0,0] + a
b = [0,0,0] + b

dp[1] = 0
dp[2] = a[2]
for i in range(3,N+1):
    dp[i] = min(dp[i-1]+a[i],dp[i-2]+b[i])

# ここから経路の計算
route = []
pos = N
while pos!= 1:
    route.append(pos)
    if dp[pos] == dp[pos-1] + a[pos]:
        pos = pos-1
    elif dp[pos] == dp[pos-2] + b[pos]:
        pos = pos-2

route.append(pos)
print(*route[::-1])

1 2 4 5
